In [1]:
import pandas as pd
import numpy as np 
from scipy.stats import norm 
import scipy.interpolate

In [2]:
#to interp rates for options
X = [0,30.41666667,60.83333333,91.25,182.5,273.75,365,730,1095,1460,1825]
Aus = [1.0449, 1.0344, 1.0138, 1.0335, 0.9978, 0.9295, 0.9044, 0.7954, 0.8295, 0.9346, 0.9839]
EUR = [-0.4132,-0.3923,-0.3919,-0.3765,-0.3627,-0.436,-0.4141,-0.4451,-0.4346,-0.4014,-0.3522]
USD = [2.1265,2.2513,2.2667,2.258,2.1328,2.0367,1.9437,1.7289,1.6493,1.6212,1.6207]
JPY = [-0.0824,-0.0991,-0.094,-0.0723,-0.03583,-0.0384,-0.0493,-0.0801,-0.0924,-0.0949,-0.09]
XCHF = [0,30.41666667,91.25,182.5,273.75,365,730,1095,1825]
CHF = [-0.7868,-0.7849,-0.7066,-0.8154,-0.8224,-0.8549,-0.8528,-0.8458,-0.7577]

In [4]:
options = pd.read_csv('Curr_opt.csv')
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,NaN,NaN,NaN
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,NaN,NaN,NaN
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,NaN,NaN,NaN
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,NaN,NaN,NaN
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,NaN,NaN,NaN


In [5]:

mats = (315, 274, 61, 122, 187)

In [6]:
rforo = []
for x in mats:
    interp = scipy.interpolate.interp1d(X, Aus, bounds_error=False, fill_value=scipy.nan)
    rforo.append(float(interp(x)))  
options['r'] = rforo    

In [7]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.918153,NaN,NaN
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.929431,NaN,NaN
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,1.013908,NaN,NaN
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,1.021470,NaN,NaN
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.994432,NaN,NaN


In [10]:
#Euro interp 
interp = scipy.interpolate.interp1d(X, EUR, bounds_error=False, fill_value=scipy.nan)
options.iloc[0,10] = interp(315)
options.iloc[1,10] = interp(274)

#USD interp
interpu = scipy.interpolate.interp1d(X, USD, bounds_error=False, fill_value=scipy.nan)
options.iloc[2,10] = interpu(61)
options.iloc[3,10] = interpu(122)

#JPY interp
interpj = scipy.interpolate.interp1d(X, JPY, bounds_error=False, fill_value=scipy.nan)
options.iloc[4,10] = interpj(187) 

In [11]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.918153,-0.426100,NaN
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.929431,-0.435940,NaN
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,1.013908,2.266652,NaN
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,1.021470,2.215809,NaN
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.994432,-0.035957,NaN


In [12]:
hist_rates = pd.read_csv('spot_rates.csv')
hist_rates.columns = ['date', 'JPY', 'USD', 'GBP', 'MYR', 'CHF', 'EUR', 'NZD']

hist_rates_ret = pd.DataFrame()

currencies = ['EUR', 'USD', 'JPY', 'CHF']

for x in currencies:
    hist_rates_ret[x] = np.log(hist_rates[x]).diff()
    
hist_rates_ret.head()

,EUR,USD,JPY,CHF
0,NaN,NaN,NaN,NaN
1,0.021320,0.005757,0.014596,0.022000
2,0.019120,0.020315,0.030557,0.024917
3,0.025635,0.001826,0.012901,0.030393
4,0.016794,0.013801,0.021336,0.017157


In [13]:
options.iloc[2, 11] = np.std(hist_rates_ret['USD']) * np.sqrt(252)
options.iloc[3, 11] = np.std(hist_rates_ret['USD']) * np.sqrt(252)
options.iloc[0,11] = np.std(hist_rates_ret['EUR']) * np.sqrt(252)
options.iloc[1,11] = np.std(hist_rates_ret['EUR']) * np.sqrt(252)
options.iloc[4,11] = np.std(hist_rates_ret['JPY']) * np.sqrt(252)

In [14]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.918153,-0.426100,0.098668
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.929431,-0.435940,0.098668
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,1.013908,2.266652,0.113376
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,1.021470,2.215809,0.113376
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.994432,-0.035957,0.142973


In [15]:
options['r'] = options['r'] / 100
options['rf'] = options['rf'] / 100

In [16]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.009182,-0.004261,0.098668
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.009294,-0.004359,0.098668
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,0.010139,0.022667,0.113376
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,0.010215,0.022158,0.113376
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.009944,-0.000360,0.142973


In [17]:
# S = spot exchange rate (one unit of foreign currency in AUD)
# K = strike price
# r = domestic rate 
# rf = foreign interest rate
# v = volatility 
# T = ttm 
def currencyoptionprice(callorput, S, K, r, rf, v, T):
    
    d1 = (np.log(S / K) + ((r-rf) + ((v * v * 0.5)) * T) / (v * np.sqrt(T)))
    
    d2 = (np.log(S / K) + ((r - rf) - (v * v * 0.5) * T ) / (v * np.sqrt(T)))
    
             
    if callorput == 'Call':
          
        optionprice = (S * np.exp(-rf * T) * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d2))
          
    elif callorput == 'Put':
        
        optionprice = (K * np.exp(-r * T) * norm.cdf(-d2)) - (S * np.exp(-rf * T) * norm.cdf(-d1))
    
    else:
        
        optionprice = 'NA'
    
    return optionprice 

#currencyoptionprice('Put', 1/0.6148, 1/0.61, 0.009182, -0.004261, (0.006215 * np.sqrt(252)), (315/365))

In [18]:
optlist = (0, 1, 2, 3, 4)
options['Price']  = ""
for x in optlist:
    ti = options.loc[x, 'Maturity'] /365
    S = 1/ (options.iloc[x, 8])
    K = 1/ options.iloc[x, 7]
    V = options.loc[x, 'vol'] #* np.sqrt(252) annualised above 
    options.iloc[x, 12] = currencyoptionprice( options.iloc[x, 3],
                                              S,
                                              K,
                                              options.loc[x, 'r'],
                                              options.loc[x, 'rf'],
                                              V,
                                              ti                                            
    
)
    

In [19]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol,Price
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.009182,-0.004261,0.098668,0.0562662
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.009294,-0.004359,0.098668,0.099668
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,0.010139,0.022667,0.113376,0.0721959
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,0.010215,0.022158,0.113376,0.0518823
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.009944,-0.000360,0.142973,0.000504728


In [20]:
options['MtM'] = options['Price'] * options['Amount'] * 1000000
#test = test * 1/options['Spot ']


In [145]:
#Delta of options 
def delta_call(S, K, T, r, y, v):
    if S>0 and K>0 and v!=0 and T > 0:
        
        term1 = np.log(S / K) + (r - y) * T
        term2 = v * v * T
        term3 = np.sqrt(term2)
        term4 = term2/2
        
        dPlus = (term1 + term4) / term3
        
        delta_call = np.exp(-y * T) * norm.cdf(dPlus)
        
        return delta_call
        
    else:
        
        delta_call = 0
        return delta_call
    
def delta_put(S, K, T, r, y, v):
    if S>0 and K>0 and v!=0 and T > 0:
        
        term1 = np.log(S / K) + (r - y) * T
        term2 = v * v * T
        term3 = np.sqrt(term2)
        term4 = term2/2
        
        dPlus = (term1 + term4) / term3
        
        delta_put = -np.exp(-y * T) * norm.cdf(-dPlus)
        
        return delta_put
        
    else:
        
        delta_put = -np.exp(-y * T)
        return delta_put 
#CALC 

options['Delta'] = ""

for x in optlist:
    Time = options.loc[x, 'Maturity'] /365
        
    if options.loc[x, 'PutCall'] == 'Call':
        options.loc[x, 'Delta'] = delta_call(1/options.loc[x, 'Spot '],
                   1/options.loc[x, 'Strike'],
                   Time,
                   options.loc[x, 'r'],
                   options.loc[x, 'rf'],
                   options.loc[x, 'vol'] #* np.sqrt(252))
    else:
        options.loc[x, 'Delta'] = delta_put(1/options.loc[x, 'Spot '],
                  1/options.loc[x, 'Strike'],
                  Time,
                  options.loc[x, 'r'],
                  options.loc[x, 'rf'],
                  options.loc[x, 'vol'] #* np.sqrt(252)) 
options                

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol,Price,Delta
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.009182,-0.004261,0.006215,0.0562662,-0.467096
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.009294,-0.004359,0.006215,0.099668,0.738899
2,USD,2-Oct-20,427,Put,Bought,120,157.8947,0.65,0.6812,0.008859,0.019072,0.007142,0.119698,-0.647579
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,0.010215,0.022158,0.007142,0.0518823,0.645899
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.009944,-0.000360,0.009006,0.000504728,0.491621


In [150]:
test2 = test * options['Delta'] #* options['Spot ']
test2

0   -3.41663e+06
1    1.03102e+07
2   -9.30166e+06
3    3.68618e+06
4    2.48135e+06
dtype: object

In [24]:
forwards = pd.read_csv('For_for_exch.csv')

In [25]:
forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,NaN,NaN
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,NaN,NaN
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,NaN,NaN
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,NaN,NaN


In [26]:
#Forward Contract
# S = spot price of one unit of cash asset
# K = contracted forward price
# r = domestic risk free rate 
# y = income flow on the asset (forein risk free rate r* when asset is foreign currency) 
# t = time to maturity 

def forward(S, K, r, y, t):
    
    f = S * np.exp(-y * t) - K * np.exp(-r * t)
    
    return f


In [40]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol,Price,MtM
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.009182,-0.004261,0.098668,0.0562662,7.31461e+06
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.009294,-0.004359,0.098668,0.099668,1.39535e+07
2,USD,2-Oct-19,61,Put,Bought,120,157.8947,0.65,0.6812,0.010139,0.022667,0.113376,0.0721959,8.6635e+06
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,0.010215,0.022158,0.113376,0.0518823,5.70705e+06
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.009944,-0.000360,0.142973,0.000504728,5.04728e+06


In [28]:
fmats = [123, 247, 276, 374]
finterp = []
for n in fmats:
    interp = scipy.interpolate.interp1d(X, Aus, bounds_error=False, fill_value=scipy.nan)
    finterp.append(float(interp(n)))  
forwards['r'] = finterp   
forwards    

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,1.021078,NaN
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.949522,NaN
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.928881,NaN
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.901712,NaN


In [29]:
#USD interp
interpu = scipy.interpolate.interp1d(X, USD, bounds_error=False, fill_value=scipy.nan)
forwards.loc[0,'y'] = interpu(123)
#EUR interp
interp = scipy.interpolate.interp1d(X, EUR, bounds_error=False, fill_value=scipy.nan)
forwards.loc[1,'y'] = interp(247)
#JPY interp
interpj = scipy.interpolate.interp1d(X, JPY, bounds_error=False, fill_value=scipy.nan)
forwards.loc[2,'y'] = interpj(276) 
#CHF interp
interpc = scipy.interpolate.interp1d(XCHF, CHF, bounds_error=False, fill_value=scipy.nan)
forwards.loc[3,'y'] = interpc(374)

forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,1.021078,2.214437
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.949522,-0.414512
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.928881,-0.038669
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.901712,-0.854848


In [30]:
forwards['r'] = forwards['r'] / 100
forwards['y'] = forwards['y'] / 100

In [31]:
fs = [0,1,2,3]
forwards['Price'] = ""
for x in fs:
    T = (forwards.loc[x, 'Maturity'] / 365)
    forwards.loc[x, 'Price']  = forward(forwards.loc[x, 'S'],
            forwards.loc[x, 'K'],
            forwards.loc[x, 'r'],
            forwards.loc[x, 'y'],
            T
                                )

forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y,Price
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,0.010211,0.022144,0.008437
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.009495,-0.004145,0.000498027
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.009289,-0.000387,-8.63332
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.009017,-0.008548,-0.0534664


In [32]:
test = forwards['Price'] / forwards['S']
test

0      0.0123855
1    0.000810064
2      -0.118313
3     -0.0793505
dtype: object

In [33]:
forwards['Vol'] = ""

forwards.loc[0, 'Vol'] = np.std(hist_rates_ret['USD']) 
forwards.loc[1, 'Vol'] = np.std(hist_rates_ret['EUR'])
forwards.loc[2, 'Vol'] = np.std(hist_rates_ret['JPY'])
forwards.loc[3, 'Vol'] = np.std(hist_rates_ret['CHF']) 
forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y,Price,Vol
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,0.010211,0.022144,0.008437,0.00714199
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.009495,-0.004145,0.000498027,0.00621549
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.009289,-0.000387,-8.63332,0.00900644
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.009017,-0.008548,-0.0534664,0.00796894


In [34]:
p = (20000000 * 1.6265 * np.exp(-(-0.00414512) * (247/365))) - (32258060* np.exp(-(0.009495) * (247/365)))
p
u = (10000000 * 1.6595 * np.exp(-0.059688*.25)) - (16500000 * np.exp(-0.049375*0.25))
u, p

def lectfor(buy, S, y, t, sell, r):
    price = ((buy*1000000) * (1 / S) * np.exp(-y *(t/365))) - ((sell*1000000) * np.exp(-r*(t/365)))
    return price 


In [35]:
for x in fs:
    
    forwards.loc[x, 'Price']  = lectfor(forwards.loc[x, 'Buy Amount '],
            forwards.loc[x, 'S'],
            forwards.loc[x, 'y'],
            forwards.loc[x, 'Maturity'],
            forwards.loc[x, 'Sell amount '],
            forwards.loc[x, 'r']
                                )
forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y,Price,Vol
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,0.010211,0.022144,-758166,0.00714199
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.009495,-0.004145,570834,0.00621549
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.009289,-0.000387,1.46517e+06,0.00900644
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.009017,-0.008548,5.53888e+06,0.00796894


In [36]:
forwards['Price'].sum()

6816712.610575922

In [37]:
forwards['exp1'] = ""
forwards['exp2'] = ""
for x in fs:
    forwards.loc[x, 'exp1'] = forwards.loc[x, 'Sell amount '] * np.exp(-(forwards.loc[x, 'r']) * (forwards.loc[x, 'Maturity']/365))
    forwards.loc[x,'exp2'] = forwards.loc[x, 'Buy Amount '] * (1 / forwards.loc[x,'S']) * np.exp(-(forwards.loc[x, 'y']) * (forwards.loc[x, 'Maturity']/365))

In [38]:
forwards

,Expiry,Maturity,Buy Currency,Buy Amount,Sell Currency,Sell amount,K,S,r,y,Price,Vol,exp1,exp2
0,3/12/2019,123,USD,25,AUD,37.31343,0.67,0.6812,0.010211,0.022144,-758166,0.00714199,37.1853,36.4271
1,5/04/2020,247,EUR,20,AUD,32.25806,0.62,0.6148,0.009495,-0.004145,570834,0.00621549,32.0514,32.6223
2,4/05/2020,276,JPY,900,AUD,10.94891,82.20,72.9700,0.009289,-0.000387,1.46517e+06,0.00900644,10.8723,12.3374
3,10/08/2020,374,CHF,35,AUD,47.29730,0.74,0.6738,0.009017,-0.008548,5.53888e+06,0.00796894,46.8623,52.4012


In [22]:
options

,Currency,Exp Date,Maturity,PutCall,BoughtSold,Amount,AUD Amount,Strike,Spot,r,rf,vol,Price,MtM
0,EUR,12-Jun-20,315,Put,Sold,130,196.9697,0.61,0.6148,0.009182,-0.004261,0.098668,0.0562662,7.31461e+06
1,EUR,2-May-20,274,Call,Bought,140,222.2222,0.64,0.6148,0.009294,-0.004359,0.098668,0.099668,1.39535e+07
2,USD,2-Oct-20,427,Put,Bought,120,157.8947,0.65,0.6812,0.008859,0.019072,0.113376,0.119698,1.43638e+07
3,USD,2-Dec-19,122,Call,Bought,110,150.6849,0.70,0.6812,0.010215,0.022158,0.113376,0.0518823,5.70705e+06
4,JPY,5-Feb-20,187,Call,Sold,10000,115.0880,72.05,72.9700,0.009944,-0.000360,0.142973,0.000504728,5.04728e+06


In [39]:
forwards['Price'].sum()

6816712.610575922

In [169]:
X_Vector = ( forwards.loc[2, 'exp2'] * 1000000,
            forwards.loc[1, 'exp2'] * 1000000,
            forwards.loc[0,'exp2'] * 1000000,
            forwards.loc[3, 'exp2'] * 1000000,
            forwards.loc[0, 'exp1'] * -1000000,
            forwards.loc[1, 'exp1'] * -1000000,
            forwards.loc[2,'exp1'] * -1000000,
            forwards.loc[3, 'exp1'] * -1000000,
            #eur 
            forwards.loc[1, 'Price'] + 
            (options.loc[0,'Price'] * options.loc[0,'Amount'] * 1000000 * options.loc[0, 'Delta']) +
            (options.loc[1,'Price'] * options.loc[1,'Amount'] * 1000000 * options.loc[1, 'Delta']),
            #USD
            forwards.loc[0, 'Price'] +
            (options.loc[2,'Price'] * options.loc[2,'Amount'] * 1000000 * options.loc[2, 'Delta']) +
            (options.loc[3,'Price'] * options.loc[3,'Amount'] * 1000000 * options.loc[3, 'Delta']),
        
            #JPY
            forwards.loc[2, 'Price'] +
            (options.loc[4,'Price'] * options.loc[4,'Amount'] * 1000000 * options.loc[4, 'Delta']),
            #CHF
            forwards.loc[3, 'Price']

)

X_Vector

(12337442.756991776,
 32622283.410038706,
 36427092.77693612,
 52401188.69995191,
 -37185259.07339107,
 -32051449.250247642,
 -10872275.75076288,
 -46862310.958941005,
 7464443.980947368,
 -6373648.89048277,
 3946513.9793219515,
 5538877.741010912)

In [41]:
options.to_excel (r'C:\Users\Mat\Documents\options.xlsx', index = None, header=True)
forwards.to_excel (r'C:\Users\Mat\Documents\forwards.xlsx', index = None, header=True)